In [1]:
from sklearn.model_selection import train_test_split
from pipeline import Pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transforms import pad_token
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
data, token_idx, idx_token = Pipeline.load("10k_common").data
ratings = Pipeline.load("ratings").data

In [3]:
class RNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, vocab_size, pad_idx):
        # RNN Accepts the following hyperparams:
        # emb_size: Embedding Size
        # hidden_size: Hidden Size of layer in RNN
        # num_layers: number of layers in RNN
        # vocab_size: vocabulary size
        super(RNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=pad_idx)
        self.rnn = nn.RNN(emb_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def init_hidden(self, batch_size):
        # Function initializes the activation of recurrent neural net at timestep 0
        # Needs to be in format (num_layers, batch_size, hidden_size)
        hidden = torch.randn(self.num_layers, batch_size, self.hidden_size)

        return hidden

    def forward(self, x, lengths):
        # reset hidden state

        batch_size, seq_len = x.size()

        self.hidden = self.init_hidden(batch_size)

        # get embedding of characters
        embed = self.embedding(x)
        # pack padded sequence
        embed = torch.nn.utils.rnn.pack_padded_sequence(embed, lengths.numpy(), batch_first=True)
        # fprop though RNN
        rnn_out, self.hidden = self.rnn(embed, self.hidden)
        # undo packing
        rnn_out, _ = torch.nn.utils.rnn.pad_packed_sequence(rnn_out, batch_first=True)
        # sum hidden activations of RNN across time
        rnn_out = torch.sum(rnn_out, dim=1)

        logits = self.linear(rnn_out)
        return logits

class ScriptsDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, key):
        return (self.data[key], self.labels[key])
    
    def get_loader(self, batch_size = 32):
        return DataLoader(
            dataset = self,
            batch_size = batch_size,
            collate_fn = self.collate,
            shuffle = True
        )
    
    def collate(self, batch):
        data_list = []
        label_list = []
        length_list = []

        for datum in batch:
            label_list.append(datum[1])
            length_list.append(len(datum[0]))
            data_list.append(torch.tensor(datum[0]))
            
        data_list = pad_sequence(data_list, batch_first = True)
        sorted_length_list, sorted_idxs = torch.sort(torch.tensor(length_list), descending = True)
        data_list = data_list[sorted_idxs]
        label_list = torch.tensor(label_list)[sorted_idxs]
        
        return data_list, sorted_length_list, label_list

In [4]:
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in tqdm(loader, desc = "Validation Batches", unit = "batch"):
        data_batch, lengths_batch, label_batch = data, lengths, labels
        predicted = model(data_batch, lengths_batch)

        total += labels.size(0)
        correct += torch.mean((predicted - labels) ** 2)
    return (correct / total)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, ratings, test_size=0.15, random_state=42)
X_train, X_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True)

train_loader = ScriptsDataset(X_train, y_train).get_loader(batch_size = 5)
val_loader = ScriptsDataset(X_test, y_test).get_loader(batch_size = 5)

In [ ]:
model = RNN(emb_size=100, hidden_size=200, num_layers=1, vocab_size=len(idx_token), pad_idx = token_idx[pad_token])

learning_rate = .1
num_epochs = 2 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)

In [ ]:
losses = []

for epoch in tqdm(range(num_epochs), desc = "Training Epochs", unit = "epoch"):
    for i, (data, lengths, labels) in enumerate(tqdm(train_loader, desc = "Batches", unit = "batch")):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(data, lengths)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Average MSE: {}'.format(
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))

Batches:  14%|█▍        | 100/704 [10:47<1:08:54,  6.84s/batch]

Validation Batches:   0%|          | 0/125 [00:00<?, ?batch/s]

Validation Batches:   1%|          | 1/125 [00:00<00:46,  2.69batch/s]

Validation Batches:   2%|▏         | 2/125 [00:00<00:43,  2.84batch/s]

Validation Batches:   2%|▏         | 3/125 [00:01<00:42,  2.84batch/s]

Validation Batches:   3%|▎         | 4/125 [00:01<00:42,  2.83batch/s]

Validation Batches:   4%|▍         | 5/125 [00:01<00:39,  3.06batch/s]

Validation Batches:   5%|▍         | 6/125 [00:01<00:37,  3.18batch/s]

Validation Batches:   6%|▌         | 7/125 [00:02<00:35,  3.32batch/s]

Validation Batches:   6%|▋         | 8/125 [00:02<00:37,  3.12batch/s]

Validation Batches:   7%|▋         | 9/125 [00:02<00:37,  3.11batch/s]

Validation Batches:   8%|▊         | 10/125 [00:03<00:36,  3.13batch/s]

Validation Batches:   9%|▉         | 11/125 [00:03<00:36,  3.09batch/s]

Validation Batches:  10%|▉         | 12/125 [00:03<00:33,  3.37batch/s]

Vali

Epoch: [1/2], Step: [101/704], Average MSE: 140868.0



Batches:  21%|██        | 147/704 [16:55<1:19:10,  8.53s/batch]